In [1]:
import os

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import perspective

from filing import Filing

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 50
plt.style.use('fivethirtyeight')

In [3]:
def percentile(n: int) -> float:
    """
    Calculates n% outcome for players, designed for use in .agg
    Example: df.groupby('name')['fpts'].agg([percentile(0), percentile(50), percentile(100)])
         --> Returns 3 columns, indexed by name corresponding to following outcomes: 0% (minimum), 50% (median), 100% (maxium)
         --> Common usage will be 25% which roughly corresponds to floor and 75% which roughly corresponds to ceiling
    """
    def percentile_(arr):
        return np.percentile(arr, n)
    # percentile_.__name__ = f'percentile_{n}'
    label = {25: 'floor', 50: 'median', 75: 'ceiling'}.get(n, f'{n}%')
    percentile_.__name__ = label
    return percentile_

In [8]:
class DataPrep:

    def __init__(self, **kwargs):
        """
        This class will contain the functionality to combine, clean, wrangle, and partition all the boxscores
        """

        self.year: int = int(kwargs.get('year', 2023))
        self.season: str = f'{self.year}-{self.year+1}'
        self.site: str = kwargs.get('site', 'draftkings').lower()

        self.name_issues = {'Gabriel Davis': 'Gabe Davis', 'Chigoziem Okonkwo': 'Chig Okonkwo', 'Josh Palmer': 'Joshua Palmer'}

        # Initialize filing object
        self.filing = Filing(self.season)

        
        self.fpts_df = self.filing.combined_boxscores()
        self.snapcounts_df = self.filing.combined_snapcounts()

        for name, replacement in self.name_issues.items():
            self.fpts_df = self.fpts_df.replace(name, replacement)
            self.snapcounts_df = self.snapcounts_df.replace(name, replacement)

        # Need to convert if necessary
        if self.site == 'fanduel':
            # HPPR and -2.0 for fumble lost
            self.fpts_df = (self.fpts_df
                            .assign(fpts=lambda df: 0.04*df.pass_yds + 4.0*df.pass_td - 1.0*df.pass_int + 0.1*df.rush_yds + 6.0*df.rush_td + 0.5*df.rec + 0.1*df.rec_yds + 6.0*df.rec_td - 2.0*df.fumbles_lost)
                           )

    def load_team_boxscores(self, team: str) -> pd.DataFrame:
        return self.filing.load_boxscores()[team]

    def load_team_snapcounts(self, team: str) -> pd.DataFrame:
        return self.filing.load_snapcounts()[team]

    def load_team_adv_stats(self, team: str, category: str) -> pd.DataFrame:
        return self.filing.load_advanced_stats()[category][team]

    # Most useful info in here
    def merge_fpts_snaps(self):

        if hasattr(self, 'fpts_snaps'):
            return self.fpts_snaps

        create_index = lambda row: f'{row.iloc[0]}-{row.iloc[1]}'

        self.fpts_df['index_'] = self.fpts_df[['name', 'week']].apply(create_index, axis=1)
        self.snapcounts_df['index_'] = self.snapcounts_df[['name', 'week']].apply(create_index, axis=1)

        # Not all people in boxscores in snapcounts, but all people in snapcounts in boxscores
        shared = self.snapcounts_df['index_'].drop_duplicates()

        # Dont want to change class dfs --> make shallow copies
        columns_to_keep = ['index_', 'name', 'team', 'opp', 'pos', 'week']
        fpts_columns_to_keep = ['pass_yds', 'pass_td', 'targets', 'rec', 'rec_yds', 'rec_td', 'rush_att', 'rush_yds', 'rush_td', 'spread', 'total', 'winner']
        fpts = (self.fpts_df
                .loc[self.fpts_df['index_'].isin(shared)]
                [columns_to_keep + fpts_columns_to_keep + ['fpts']]
                .sort_values('index_')
                .set_index('index_')
               )

        snaps = (self.snapcounts_df
                 [columns_to_keep + ['snap_total', 'snap_percent']]
                 .sort_values('index_')
                 .set_index('index_')
                )

        self.fpts_snaps = fpts.merge(snaps)

        # defense_path = os.path.join(self.filing.season_dir, 'defenses.csv')
        # defense_df = (pd
        #               .read_csv(defense_path)
        #               .set_index('team')
        #              )

        # self.fpts_snaps['opp-defense'] = self.fpts_snaps['opp'].map(lambda opp_team: f'vs{defense_df.loc[opp_team, "defense"]}')
        
        return self.fpts_snaps

    def get_pos_depths(self):

        if hasattr(self, 'pos_depths'):
            return self.pos_depths

        # Reference to self.fpts_snaps, need to figure out injury stuff --> AJ Dillon comes out as RB1 everyway except fpts rn
        df = (self.merge_fpts_snaps()
              # .pipe(lambda df_: df_.loc[df_['fpts'] > 0.0])
             )

        self.pos_depths = {
            team: {
                pos: dict()
                for pos in df['pos'].drop_duplicates()
            }
            for team in df['team'].drop_duplicates()
        }

        # Info to get for each position
        # by --> What to determine depth by, becomes more accurate with regression / increased sample size
        # depth --> amount of players to care about
        # agg --> method to determine by
        pos_depth_info = {
            'QB': {
                'by': 'snap_total',
                'depth': range(1,2)
            },
            'WR': {
                'by': 'targets',
                'depth': range(1,4)
            },
            'RB': {
                'by': 'snap_total',
                'depth': range(1,3)
            },
            'TE': {
                'by': 'targets',
                'depth': range(1,3)
            }
        }

        for team in self.pos_depths:
            # Team dataframe
            tdf = df.loc[df['team'] == team]

            for pos in pos_depth_info:

                info = pos_depth_info[pos]

                determine_by = info.get('by', 'snap_total')
                agg_by = info.get('agg', 'sum')
                
                # Team position dataframe
                tpdf = tdf.loc[tdf['pos'] == pos]
                # Agg default is sum
                aggdf = tpdf.groupby('name')[determine_by].agg([agg_by]).sort_values(agg_by, ascending=False)

                names = tuple(aggdf.index)

                for i, name in enumerate(names):
                    depth_ = i+1
                    if depth_ in info['depth']:
                        self.pos_depths[team][pos][depth_] = name
                    else:
                        depth_ = list(info['depth'])[-1]+1
                        if depth_ in self.pos_depths[team][pos]:
                            self.pos_depths[team][pos][depth_].append(name)
                        else:
                            self.pos_depths[team][pos][depth_] = [name]
            
        
        return self.pos_depths

    def add_depths(self):

        pos_depths = self.get_pos_depths()

        # Returns itself if already defined, creates itself if not
        df = self.merge_fpts_snaps().copy(deep=True)

        for team, pos_info in pos_depths.items():
            # Team: {pos: {n: name, ...}}
            for pos, depths in pos_info.items():
                # Get the top players as defined in pos_depth_info above, aka ones not in list
                top_n_players = {depth_: name_ for depth_, name_ in depths.items() if isinstance(name_, str)}
                # Need better naming --> this is the default for lower depth players (4 for WR, 2 for QB etc)
                bad_players_depth = max(depths.keys())

                for depth_, name_ in top_n_players.items():
                    pos_depth_value = f'{pos}{depth_}'
                    df.loc[(df['team'] == team) & (df['pos'] == pos) & (df['name'] == name_), 'depth'] = pos_depth_value

                for name_ in depths[bad_players_depth]:
                    pos_depth_value = f'{pos}{bad_players_depth}'
                    df.loc[(df['team'] == team) & (df['pos'] == pos) & (df['name'] == name_), 'depth'] = pos_depth_value
                    

        self.fpts_snaps = df
        
        return None

    def load_analysis_data(self):
        if not hasattr(self, 'fpts_snaps'):
            self.add_depths()

        df = self.fpts_snaps
        df['abs-spread'] = abs(df['spread'])

        return df

    def advanced_receiving(self, **kwargs):
        
        if not hasattr(self, 'fpts_snaps'):
            self.add_depths()

        rec_columns = ['week', 'name', 'team', 'opp', 'pos', 'depth', 'fpts', 'snap_total', 'snap_percent', 'targets', 'rec', 'rec_yds', 'rec_td', 'opp-defense', 'spread', 'total', 'winner']
        rec_pos = ['WR', 'TE', 'RB']
        
        df = (self.fpts_snaps
              .pipe(lambda df_: df_.loc[df_['pos'].isin(rec_pos)])
              [rec_columns]
             )

        adv_rec = (pd
                   .concat(self.filing.load_advanced_stats()['receiving'].values())
                   .pipe(lambda df_: df_.loc[df_['pos'].isin(rec_pos)])
                  )

        create_index = lambda row: f'{row.iloc[0]}-{row.iloc[1]}'

        df['index_'] = df[['name', 'week']].apply(create_index, axis=1)
        adv_rec['index_'] = adv_rec[['name', 'week']].apply(create_index, axis=1)

        df = df.set_index('index_')
        adv_rec = adv_rec.set_index('index_')

        return df.merge(adv_rec)

    def advanced_rushing(self, **kwargs):
        
        if not hasattr(self, 'fpts_snaps'):
            self.add_depths()

        rush_columns = ['week', 'name', 'team', 'opp', 'pos', 'depth', 'fpts', 'snap_total', 'snap_percent', 'rush_att', 'rush_yds', 'rush_td', 'opp-defense', 'spread', 'total', 'winner']
        rush_pos = ['QB', 'RB']
        
        df = (self.fpts_snaps
              .pipe(lambda df_: df_.loc[df_['pos'].isin(rush_pos)])
              [rush_columns]
             )

        adv_rush = (pd
                   .concat(self.filing.load_advanced_stats()['rushing'].values())
                   .pipe(lambda df_: df_.loc[df_['pos'].isin(rush_pos)])
                  )

        create_index = lambda row: f'{row.iloc[0]}-{row.iloc[1]}'

        df['index_'] = df[['name', 'week']].apply(create_index, axis=1)
        adv_rush['index_'] = adv_rush[['name', 'week']].apply(create_index, axis=1)

        df = df.set_index('index_')
        adv_rush = adv_rush.set_index('index_')

        return df.merge(adv_rush)

    def advanced(self, category: str, **kwargs):
        return {
            'rushing': self.advanced_rushing,
            'receiving': self.advanced_receiving,
        }[category](**kwargs)

    def load_team_receiving(self, team: str) -> pd.DataFrame:

        return (self.advanced('receiving')
                .pipe(lambda df_: df_.loc[df_['team'] == team])
                .groupby('name')
                [['fpts', 'snap_percent', 'targets', 'rec_adot', 'rec_td']]
                .agg(['count', percentile(25), percentile(50), percentile(75), 'sum'])
                .sort_values(('targets', 'sum'), ascending=False)
                .drop([('snap_percent', 'count'), ('targets', 'count'), ('rec_adot', 'count')], axis=1)
                .round(2)
               )

    def load_team_rushing(self, team: str) -> pd.DataFrame:

        return (self.advanced('rushing')
                .pipe(lambda df_: df_.loc[df_['team'] == team])
                .groupby('name')
                [['fpts', 'snap_percent', 'rush_att', 'rush_yds', 'rush_td']]
                .agg(['count', percentile(25), percentile(50), percentile(75), 'sum'])
                .sort_values(('rush_att', 'sum'), ascending=False)
                .drop([('snap_percent', 'count'), ('rush_att', 'count'), ('rush_yds', 'count'), ('rush_td', 'count')], axis=1)
                .round(2)
               )

    def create_median_projections(self, game: str) -> pd.DataFrame:
        """
        Creates median projections for a given game, need to enter as away-home
        """
        df = (self.fpts_df
              .pipe(lambda df_: df_.loc[df_['team'].isin(game.split('-'))])
             )

        return (df
                .groupby('name')
                ['fpts']
                .agg(['median']) # want as dataframe, hence []
                .set_axis(['fpts'], axis=1)
                .sort_values('fpts', ascending=False)
               )

    def clean_name(self, name: str) -> str:
        """
        Standardizes name across PFR, FD, DK
        """
        clean_ = ' '.join(name.split(' ')[:2]).replace('.', '')
        return self.name_issues.get(clean_, clean_)


    def create_contest_data(self, game: str, **kwargs) -> pd.DataFrame:

        contest_columns = {
            'draftkings': {
                'Name': 'name',
                'Salary': 'salary',
                'Position': 'pos',
                'TeamAbbrev': 'team'
            },
        }

        current_data_path = os.path.join(self.filing.season_dir, 'contest-files', self.site, 'current', f'{game}.csv')
        injured = kwargs.get('injured', list())

        medians_df = self.create_median_projections(game)
        contest_df = (pd
                      .read_csv(current_data_path)
                      .pipe(lambda df_: df_.loc[df_['Roster Position'] != 'CPT', list(contest_columns[self.site].keys())])
                      .rename(contest_columns[self.site], axis=1)
                      .assign(name=lambda df_: df_.name.str.strip())
                      .set_index('name')
                     )

        contest_df.index = contest_df.index.map(lambda name_: self.clean_name(name_))
        
        optimizer_data = (pd
                          .concat([medians_df, contest_df], axis=1)
                          .fillna(0.0) # Careful
                          .assign(fpts_1k=lambda df_: 1000 * df_.fpts / df_.salary)
                          # .fillna(0.0)
                          .round(2)
                         )


        edits = {
            **{name: 0.0 for name in kwargs.get('inactive', list())},
            **kwargs.get('edit', dict())
        }

        for name, fpts in edits.items():
            optimizer_data.loc[name, 'fpts'] = fpts


        pool = kwargs.get('pool', list(optimizer_data.index))
        
        optimizer_data = (optimizer_data
                          .loc[optimizer_data.index.isin(pool)]
                          .dropna()
                          .assign(
                              fpts_1k=lambda df_: 1000 * df_.fpts / df_.salary,
                              salary=lambda df_: df_.salary.astype('int'),
                          )
                          .sort_values('fpts', ascending=False)
                         )

        print(f'{len(optimizer_data.loc[optimizer_data["fpts"] > 0.0].index)} total players projected currently...\n')
        
        optimizer_data.to_csv(current_data_path.replace('current', 'projections')) #, index=False)

        return optimizer_data

In [36]:
dataprep = DataPrep(
    year=2023,
)

In [37]:
df_2022 = DataPrep(year=2022).load_analysis_data()

In [41]:
COMPARE = ['Justin Herbert', 'Austin Ekeler', 'Keenan Allen']

In [42]:
weeks_together = tuple(df_2022
                       .pipe(lambda df_: df_.loc[(df_['team'] == 'LAC') & (df_['name'].isin(COMPARE))])
                       .groupby('week')
                       ['name']
                       .agg(['count'])
                       .pipe(lambda df_: df_.loc[df_['count'] == 3] )
                       .index
                      )

In [ ]:
jh_ae_weeks_together = tuple(df_2022
                             .pipe(lambda df_: df_.loc[(df_['team'] == 'LAC') & (df_['name'].isin(['Austin Ekeler', 'Justin Herbert']))])
                             .groupby('week')
                             ['name']
                             .agg(['count'])
                             .pipe(lambda df_: df_.loc[df_['count'] == 2] )
                             .index
                             )

In [44]:
comp = (df_2022
        .pipe(lambda df_: df_.loc[(df_['team'] == 'LAC') & (df_['name'].isin(COMPARE)) & (df_['week'].isin(weeks_together))])
        .sort_values('week')
        [['name', 'team', 'opp', 'week', 'targets', 'fpts']]
       )

comp

,name,team,opp,week,targets,fpts
318,Austin Ekeler,LAC,LV,1,4.0,11.20
3074,Keenan Allen,LAC,LV,1,4.0,10.60
2898,Justin Herbert,LAC,LV,1,0.0,23.26
2913,Justin Herbert,LAC,SEA,7,1.0,20.12
333,Austin Ekeler,LAC,SEA,7,12.0,36.70
3083,Keenan Allen,LAC,SEA,7,2.0,3.10
2900,Justin Herbert,LAC,KC,11,1.0,20.70
320,Austin Ekeler,LAC,KC,11,2.0,18.00
3075,Keenan Allen,LAC,KC,11,8.0,13.40
2901,Justin Herbert,LAC,ARI,12,0.0,26.76


In [45]:
ae = comp.loc[comp['name'] == 'Austin Ekeler', 'fpts'].reset_index(drop=True)
ka = comp.loc[comp['name'] == 'Keenan Allen', 'fpts'].reset_index(drop=True)
jh = comp.loc[comp['name'] == 'Justin Herbert', 'fpts'].reset_index(drop=True)

ae_ka_corr = np.corrcoef(ae, ka)[0,1]
ae_jh_corr = np.corrcoef(ae, jh)[0,1]
jh_ka_corr = np.corrcoef(jh, ka)[0,1]

print(*[
    f'Austin Ekeler--Keenan Allen correlation: {ae_ka_corr}',
    f'Austin Ekeler--Justin Herbert correlation: {ae_jh_corr}',
    f'Justin Herbert--Keenan Allen correlation: {jh_ka_corr}',
], sep='\n')

Austin Ekeler--Keenan Allen correlation: -0.5835111153571583
Austin Ekeler--Justin Herbert correlation: 0.00550935507191786
Justin Herbert--Keenan Allen correlation: -0.26702443540278037


In [6]:
CURRENT = [
    'DAL',
    'LAC'
]

In [7]:
(dataprep.fpts_df
 .pipe(lambda df_: df_.loc[(df_['pos'] == 'QB') & (df_['fpts'] > 2.0) & (df_['opp'].isin(CURRENT))])
 [['name', 'pos', 'team', 'opp', 'fpts', 'pass_att', 'pass_yds', 'pass_td']]
 .groupby('opp')
 [['fpts', 'pass_att', 'pass_yds', 'pass_td']]
 .agg([percentile(25), 'median', percentile(75), 'max', 'sum'])
)

fpts                              pass_att                              \
      floor median ceiling    max    sum    floor median ceiling   max    sum   
opp                                                                             
DAL   4.390   8.93  15.645  26.08  66.98    21.00   22.5   26.25  28.0  130.0   
LAC  18.985  25.36  30.295  32.14  95.68    35.25   42.0   46.25  50.0  158.0   

    pass_yds                               pass_td                           
       floor median ceiling    max     sum   floor median ceiling  max  sum  
opp                                                                          
DAL    115.5  160.0  184.25  252.0   922.0    0.00    0.5     1.0  4.0  6.0  
LAC    244.0  306.5  391.75  466.0  1317.0    0.75    2.0     3.0  3.0  7.0

In [8]:
# fpts = dataprep.fpts_df
fpts = dataprep.load_analysis_data()

rec_pos = ['WR']
rb_stats = ['fpts', 'rush_att', 'rush_yds', 'rush_td', 'targets', 'rec', 'rec_yds', 'rec_td']
stats = ['snap_total', 'targets', 'rec', 'rec_yds', 'rec_td', 'fpts']
wr_stats = ['fpts', 'targets', 'rec', 'rec_yds', 'rec_td']

rec = (fpts
       .loc[(fpts['pos'].isin(rec_pos)) & (fpts['opp'].isin(CURRENT)) & (fpts['depth'].isin([f'{rec_pos[0]}{n}' for n in range(1,3)]))]
       [['name', 'pos', 'team', 'opp', 'snap_total'] + wr_stats] # 'fpts', 'targets', 'rec', 'rec_yds', 'rec_td']]
      )

(rec
 .groupby('opp')
 [wr_stats + ['snap_total']]
 # [['fpts', 'targets', 'rec_yds', 'rec_td']]
 .agg([percentile(25), 'median', percentile(75), 'max', 'sum'])
 .sort_values(('fpts', 'sum'), ascending=False)
 .round(2)
)

fpts                             targets                             \
    floor median ceiling   max    sum   floor median ceiling   max   sum   
opp                                                                        
LAC  10.4  11.65   19.35  47.5  141.7    4.75    6.5    13.0  15.0  66.0   
DAL   4.5   9.80   16.20  17.1   87.7    4.00    5.0     7.0   8.0  48.0   

      rec                            rec_yds                               \
    floor median ceiling   max   sum   floor median ceiling    max    sum   
opp                                                                         
LAC  3.75    5.0    7.25  11.0  45.0   38.25   63.5   95.75  215.0  667.0   
DAL  2.00    3.0    4.00   5.0  26.0   15.00   36.0   58.00   83.0  353.0   

    rec_td                          snap_total                          
     floor median ceiling  max  sum      floor median ceiling max  sum  
opp                                                                     
LAC    0.0    0.0     1.0  2.0  4.0      43.75   51.0   64.75  78  439  
DAL    0.0    0.0     0.0  1.0  2.0      33.00   44.0   52.00  58  388

In [9]:
# fpts = dataprep.fpts_df
fpts = dataprep.load_analysis_data()

pos_ = ['RB']
rb_stats = ['fpts', 'rush_att', 'rush_yds', 'rush_td', 'targets', 'rec', 'rec_yds', 'rec_td']
stats = ['snap_total', 'targets', 'rec', 'rec_yds', 'rec_td', 'fpts']
wr_stats = ['fpts', 'targets', 'rec', 'rec_yds', 'rec_td']

rb_ = (fpts
       .loc[(fpts['pos'].isin(pos_)) & (fpts['opp'].isin(CURRENT))]
       [['name', 'pos', 'team', 'opp', 'snap_total'] + rb_stats]
      )

(rb_
 .groupby('opp')
 [rb_stats + ['snap_total']]
 # [['fpts', 'targets', 'rec_yds', 'rec_td']]
 .agg([percentile(25), 'median', percentile(75), 'max', 'sum'])
 .sort_values(('fpts', 'sum'), ascending=False)
 .round(2)
)

fpts                            rush_att                              \
    floor median ceiling   max   sum    floor median ceiling   max    sum   
opp                                                                         
LAC  3.40    7.5   17.50  27.9  94.5      3.0    8.0    17.0  25.0   89.0   
DAL  1.72    3.1    8.48  19.6  81.1      2.5    5.5    11.5  19.0  101.0   

    rush_yds                             rush_td                           \
       floor median ceiling   max    sum   floor median ceiling  max  sum   
opp                                                                         
LAC    15.00   37.0   58.00  93.0  370.0     0.0    0.0     1.0  1.0  3.0   
DAL     8.25   14.5   45.75  98.0  389.0     0.0    0.0     0.0  1.0  3.0   

    targets                              rec                            \
      floor median ceiling   max   sum floor median ceiling  max   sum   
opp                                                                      
LAC     2.0    2.0    4.00  11.0  33.0  1.00    2.0     3.0  8.0  23.0   
DAL     1.0    2.0    2.75   4.0  25.0  0.25    1.0     2.0  3.0  17.0   

    rec_yds                             rec_td                           \
      floor median ceiling   max    sum  floor median ceiling  max  sum   
opp                                                                       
LAC     4.0   13.0    15.0  81.0  165.0    0.0    0.0     0.0  0.0  0.0   
DAL     0.0    5.5     9.5  27.0   92.0    0.0    0.0     0.0  0.0  0.0   

    snap_total                          
         floor median ceiling max  sum  
opp                                     
LAC       15.0   24.0   49.00  66  289  
DAL       11.5   16.0   31.75  52  308

In [11]:
dst =(dataprep.fpts_df
      .pipe(lambda df_: df_.loc[df_['pos'] == 'DST'])
      [['name', 'team', 'opp', 'winner', 'spread', 'total', 'fpts']]
     )
dst.loc[dst['team'].isin(CURRENT)]

,name,team,opp,winner,spread,total,fpts
98,Cowboys,DAL,NYJ,1,20,40,15.0
460,Chargers,LAC,TEN,0,-3,51,5.0
621,Cowboys,DAL,NYG,1,40,40,29.0
1384,Cowboys,DAL,ARI,0,-12,44,1.0
1462,Chargers,LAC,MIA,0,-2,70,0.0
1672,Cowboys,DAL,NE,1,35,41,27.0
1881,Chargers,LAC,LV,1,7,41,14.0
2214,Chargers,LAC,MIN,1,4,52,8.0
2348,Cowboys,DAL,SF,0,-32,52,-1.0


In [10]:
# rec = dataprep.advanced('receiving')
dataprep.load_team_receiving('LAC')

fpts                               snap_percent         \
                 count  floor median ceiling     sum        floor median   
name                                                                       
Keenan Allen         4  13.70  24.15   37.69  108.96         0.86   0.90   
Mike Williams        3  12.55  16.60   22.35   53.20         0.72   0.77   
Josh Palmer          4   3.58   7.50   11.92   32.00         0.59   0.62   
Gerald Everett       4   3.95   6.00    8.02   23.90         0.54   0.63   
Quentin Johnston     4   2.52   2.85    2.92   10.40         0.22   0.26   
Donald Parham        4   1.28   6.45   12.00   27.30         0.42   0.50   
Austin Ekeler        1  29.40  29.40   29.40   29.40         0.51   0.51   
Derius Davis         3   1.40   1.50    5.10   11.50         0.06   0.09   
Joshua Kelley        3   3.30   3.90    9.50   21.70         0.60   0.73   
Elijah Dotson        1   3.90   3.90    3.90    3.90         0.13   0.13   
Isaiah Spiller       1   4.70   4.70    4.70    4.70         0.23   0.23   
Stone Smartt         2   0.85   1.70    2.55    3.40         0.34   0.36   

                               targets                      rec_adot         \
                 ceiling   sum   floor median ceiling   sum    floor median   
name                                                                          
Keenan Allen        0.92  3.49    8.00    9.5   12.50  44.0     9.57  11.20   
Mike Williams       0.84  2.34    6.50    8.0   10.50  26.0    10.35  12.30   
Josh Palmer         0.70  2.65    4.00    6.0    7.25  21.0     7.50  12.15   
Gerald Everett      0.68  2.36    2.75    3.0    3.75  14.0     1.80   2.15   
Quentin Johnston    0.33  1.17    2.75    3.0    3.00  11.0     7.30  10.75   
Donald Parham       0.50  1.74    1.75    2.0    2.25   8.0     3.25   9.50   
Austin Ekeler       0.51  0.51    5.00    5.0    5.00   5.0     4.60   4.60   
Derius Davis        0.14  0.31    1.00    1.0    1.50   4.0    -5.50  -5.00   
Joshua Kelley       0.76  2.00    1.00    1.0    1.00   3.0     1.50   4.00   
Elijah Dotson       0.13  0.13    2.00    2.0    2.00   2.0    -0.50  -0.50   
Isaiah Spiller      0.23  0.23    2.00    2.0    2.00   2.0    -0.50  -0.50   
Stone Smartt        0.38  0.72    1.00    1.0    1.00   2.0    17.25  18.50   

                               rec_td                            
                 ceiling   sum  count floor median ceiling  sum  
name                                                             
Keenan Allen       12.02  41.6      4   0.0    0.5    1.25  3.0  
Mike Williams      12.70  33.8      3   0.0    0.0    0.50  1.0  
Josh Palmer        16.12  45.9      4   0.0    0.0    0.25  1.0  
Gerald Everett      3.40  12.2      4   0.0    0.0    0.00  0.0  
Quentin Johnston   13.42  39.9      4   0.0    0.0    0.00  0.0  
Donald Parham      15.12  35.5      4   0.0    0.5    1.25  3.0  
Austin Ekeler       4.60   4.6      1   0.0    0.0    0.00  0.0  
Derius Davis       -4.50 -15.0      3   0.0    0.0    0.00  0.0  
Joshua Kelley       4.50   8.0      3   0.0    0.0    0.00  0.0  
Elijah Dotson      -0.50  -0.5      1   0.0    0.0    0.00  0.0  
Isaiah Spiller     -0.50  -0.5      1   0.0    0.0    0.00  0.0  
Stone Smartt       19.75  37.0      2   0.0    0.0    0.00  0.0

In [24]:
dataprep.load_team_receiving('DAL')

fpts                             snap_percent         \
                  count  floor median ceiling   sum        floor median   
name                                                                      
CeeDee Lamb           5  10.20  11.70   14.80  73.9         0.69   0.72   
Jake Ferguson         5   5.80   9.80   10.10  43.5         0.60   0.69   
Michael Gallup        5   2.00   3.50   11.00  33.0         0.67   0.69   
Tony Pollard          5   9.40  17.90   18.10  76.6         0.64   0.64   
Brandin Cooks         4   3.35   3.95    4.82  16.9         0.64   0.68   
Jalen Tolbert         3   2.40   4.80    7.05  14.1         0.31   0.34   
KaVontae Turpin       4   5.20   7.85    9.82  28.7         0.14   0.18   
Luke Schoonmaker      4   0.00   0.00    1.78   7.1         0.33   0.38   
Rico Dowdle           4   2.40   4.00    7.38  23.1         0.13   0.17   
Deuce Vaughn          3   1.95   2.10    4.15  10.1         0.12   0.15   
Peyton Hendershot     2   0.32   0.65    0.98   1.3         0.46   0.47   
Hunter Luepke         1   3.10   3.10    3.10   3.1         0.10   0.10   

                                targets                      rec_adot         \
                  ceiling   sum   floor median ceiling   sum    floor median   
name                                                                           
CeeDee Lamb          0.79  3.66     5.0    6.0    7.00  35.0     7.40   9.00   
Jake Ferguson        0.69  3.26     4.0    7.0    7.00  28.0     1.70   4.00   
Michael Gallup       0.72  3.43     2.0    5.0    6.00  22.0     9.00  10.10   
Tony Pollard         0.69  3.36     3.0    3.0    5.00  22.0    -0.70   0.40   
Brandin Cooks        0.72  2.74     4.0    4.0    4.75  19.0     6.75   8.70   
Jalen Tolbert        0.51  1.30     2.5    4.0    4.00   9.0    11.65  13.00   
KaVontae Turpin      0.22  0.71     1.0    1.0    1.50   6.0     2.68   3.00   
Luke Schoonmaker     0.45  1.56     1.0    1.0    1.50   6.0     0.75   4.00   
Rico Dowdle          0.20  0.64     1.0    1.0    1.50   6.0    -4.03  -0.85   
Deuce Vaughn         0.23  0.56     1.0    1.0    2.00   5.0    -2.00   0.00   
Peyton Hendershot    0.48  0.94     1.0    1.0    1.00   2.0     5.00   9.00   
Hunter Luepke        0.10  0.10     1.0    1.0    1.00   1.0     3.00   3.00   

                                rec_td                            
                  ceiling   sum  count floor median ceiling  sum  
name                                                              
CeeDee Lamb          9.00  41.9      5   0.0    0.0    0.00  1.0  
Jake Ferguson        4.10  17.5      5   0.0    0.0    0.00  1.0  
Michael Gallup      17.60  59.2      5   0.0    0.0    0.00  0.0  
Tony Pollard         0.90  -0.4      5   0.0    0.0    0.00  0.0  
Brandin Cooks       15.12  52.7      4   0.0    0.0    0.00  0.0  
Jalen Tolbert       16.50  43.3      3   0.0    0.0    0.00  0.0  
KaVontae Turpin      8.75  33.7      4   0.0    0.0    0.25  1.0  
Luke Schoonmaker    10.00  27.0      4   0.0    0.0    0.25  1.0  
Rico Dowdle          2.50  -2.7      4   0.0    0.0    0.25  1.0  
Deuce Vaughn         0.50  -3.0      3   0.0    0.0    0.00  0.0  
Peyton Hendershot   13.00  18.0      2   0.0    0.0    0.00  0.0  
Hunter Luepke        3.00   3.0      1   0.0    0.0    0.00  0.0

In [ ]:
dataprep.load_team_rushing('NE')

In [ ]:
def percentile(n: int) -> float:
    """
    Calculates n% outcome for players, designed for use in .agg
    Example: df.groupby('name')['fpts'].agg([percentile(0), percentile(50), percentile(100)])
         --> Returns 3 columns, indexed by name corresponding to following outcomes: 0% (minimum), 50% (median), 100% (maxium)
         --> Better use would be 25% which roughly corresponds to floor and 75% which roughly corresponds to ceiling
    """
    def percentile_(arr):
        return np.percentile(arr, n)
    # percentile_.__name__ = f'percentile_{n}'
    label = {25: 'floor', 75: 'ceiling'}.get(n, f'{n}%')
    percentile_.__name__ = label
    return percentile_

In [52]:
df = dataprep.load_analysis_data()

In [ ]:
rb_stats = ['snap_total', 'rush_att', 'rush_yds', 'rush_td', 'targets', 'rec', 'rec_yds', 'rec_td', 'fpts']
stats = ['snap_total', 'targets', 'rec', 'rec_yds', 'rec_td', 'fpts']
wr_stats = ['fpts', 'targets', 'rec', 'rec_yds', 'rec_td']
agg_stats = ['count', 'mean', 'sum']
new_columns = [f'{agg_}-{stat_}' for stat_ in wr_stats for agg_ in agg_stats]

In [ ]:
(dataprep.fpts_df
 .loc[dataprep.fpts_df['pos'] == 'WR']
 .groupby('name')
 [wr_stats]
 .agg(agg_stats)
 .set_axis([f'{agg_}-{stat_}' for stat_ in wr_stats for agg_ in agg_stats], axis=1)
 .round(2)
 .drop([col for col in new_columns if 'count' in col and col != 'count-fpts'], axis=1).rename({'count-fpts': 'games'}, axis=1).sort_values('mean-fpts', ascending=False).head(100)
)

In [ ]:
def_stats = ['pass_sacked', 'pass_int']
agg_stats = ['count', 'sum', 'median']
new_columns = [f'{agg_}-{stat_}' for stat_ in def_stats for agg_ in agg_stats]
# new_columns
drop_columns = ['count-pass_int']
rename_columns = {'count-pass_sacked': 'games'}

In [ ]:
sack_leaders = (dataprep.fpts_df
                .pipe(lambda df_: df_.loc[(df_['pos'] == 'QB') & (df_['fpts'] >= 3.0)])
                # .assign(pass_sacked=lambda df_: df_.pass_sacked.astype('uint8'))
                .groupby('name')
                [def_stats]
                .agg(agg_stats)
                .set_axis(new_columns, axis=1)
                .assign(**{c: lambda df_, c=c:df_[c].astype('uint8') for c in new_columns})
                .drop(drop_columns, axis=1)
                .rename(rename_columns, axis=1)
                .pipe(lambda df_: df_.loc[df_['games'] > 2])
               )

replacements = {'pass_sacked': 'sacks', 'pass_int': 'ints', 'sum': 'total'}
for part, replace_ in replacements.items():
    sack_leaders.columns = sack_leaders.columns.str.replace(part, replace_)
# sack_leaders.columns = sack_leaders.columns.str.replace('pass_sacked', 'sacks')

sack_leaders.sort_values('total-sacks', ascending=False)

| Unique Players | Values |
| -------- | ------- |
| mean |     10.33 |
| std |       1.36 |
| min |       8 |
| 25% |       9 |
| 50% |     10 |
| 75% |     11 |
| max |      13 |

***Therefore, really want to narrow down to 13 players at most***

In [86]:
pool = [
    'Austin Ekeler',
    'Justin Herbert',
    'Keenan Allen',
    'Joshua Palmer',
    'Tony Pollard',
    'Dak Prescott',
    'CeeDee Lamb',
    'Jake Ferguson',
    'KaVontae Turpin',
    'Jalen Tolbert',
    'Luke Schoonmaker',
    'Michael Gallup',
    'Brandin Cooks'
]
print(f'{len(pool)} players in input pool\n')

13 players in input pool



In [131]:
game = 'DAL-LAC'

# Eliminations from player pool, add as inactive
inactive = [
    'Mike Williams',
    'Cooper Rush',

]
edit = {
    'Austin Ekeler': 25.55,
    'Luke Schoonmaker': 3.5,
    'Joshua Palmer': 9.5,
    'Justin Herbert': 19.0,
    'CeeDee Lamb': 17.0,
    'KaVontae Turpin': 3.5,
    'Dak Prescott': 12.3,
    'Brandin Cooks': 8.5,
    'Jalen Tolbert': 3.5,
    'Michael Gallup': 4.0,
}

In [132]:
# dataprep.create_median_projections('DET-GB') #.reset_index()
dataprep.create_contest_data(
    game,
    pool=pool,
    # inactive=inactive,
    edit=edit
).pipe(lambda df_: df_.loc[df_['fpts'] > 0.0]).round(2)

13 total players projected currently...



,fpts,salary,pos,team,fpts_1k
name,,,,,
Austin Ekeler,25.55,11200,RB,LAC,2.28
Keenan Allen,24.15,10600,WR,LAC,2.28
Justin Herbert,19.00,10800,QB,LAC,1.76
Tony Pollard,17.90,9600,RB,DAL,1.86
CeeDee Lamb,17.00,9000,WR,DAL,1.89
Dak Prescott,12.30,10000,QB,DAL,1.23
Jake Ferguson,9.80,5200,TE,DAL,1.88
Joshua Palmer,9.50,7000,WR,LAC,1.36
Brandin Cooks,8.50,5400,WR,DAL,1.57


In [115]:
def analyze_defenses(against: str) -> pd.DataFrame:
    """
    Input is team abbreviation not full name: GB not Packers
    Little confusing so better with example:
    analyze_defenses(against=LV):
        - Pulls up opposing QBs and defense stat lines
    """
    df = dataprep.fpts_df

    # Will not deal with fumbles for rn
    info_cols = ['name', 'team', 'opp', 'pos', 'fpts']
    qb_cols = ['pass_int', 'pass_sacked']

    columns = sum([
        info_cols,
        qb_cols,
    ], [])

    return (df
            .pipe(lambda df_: df_.loc[(df_['fpts'] > 0.0) | (df_['pos'] == 'DST')])
            .loc[((df['opp'] == against) & (df['pos'].isin(['DST', 'QB'])))| ((df['team'] == against) & (df['pos'] == 'QB'))]
            [columns]
            .assign(**{c: lambda df_, c=c:df_[c].astype('int') for c in qb_cols})
            .sort_values('opp')
            .reset_index(drop=True)
           )

def analyze_kickers(against: str) -> pd.DataFrame:
    """
    Input is team abbreviation not full name: GB not Packers
    Little confusing so better with example:
    analyze_kickers(against=LV):
        - Pulls up opposing Ks stat lines
    """
    df = dataprep.fpts_df

    # Will not deal with fumbles for rn
    info_cols = ['name', 'team', 'opp', 'pos', 'fpts']
    # qb_cols = ['pass_int', 'pass_sacked']

    columns = sum([
        info_cols,
        # qb_cols,
    ], [])

    return (df
            # .pipe(lambda df_: df_.loc[(df_['fpts'] > 0.0) & (df_['pos'] == 'K')])
            .loc[(df['opp'] == against) & (df['pos'].isin(['K'])) & (df['fpts'] > 0.0)] #| ((df['team'] == against) & (df['pos'] == 'QB'))]
            [columns]
            # .assign(**{c: lambda df_, c=c:df_[c].astype('int') for c in qb_cols})
            .sort_values('opp')
            .reset_index(drop=True)
           )

In [116]:
pd.options.display.max_rows = 150

In [26]:
(pd
 .concat([analyze_defenses(against_) for against_ in CURRENT]) #game.split('-')])
 .pipe(lambda df_: df_.loc[df_['pos'] == 'DST'])
 .sort_values(['opp', 'fpts'])
 # .groupby('opp')
 # ['fpts']
 # .agg([percentile(25), 'median', percentile(75), 'max', 'sum'])
 # .sort_values('median', ascending=False)
)

,name,team,opp,pos,fpts,pass_int,pass_sacked
11,Giants,NYG,DAL,DST,-1.0,0,0
10,Jets,NYJ,DAL,DST,0.0,0,0
2,Patriots,NE,DAL,DST,3.0,0,0
7,Cardinals,ARI,DAL,DST,5.0,0,0
12,49ers,SF,DAL,DST,16.0,0,0
3,Dolphins,MIA,LAC,DST,2.0,0,0
7,Vikings,MIN,LAC,DST,2.0,0,0
1,Titans,TEN,LAC,DST,3.0,0,0
5,Raiders,LV,LAC,DST,4.0,0,0


In [21]:
(pd
 .concat([analyze_defenses(against_) for against_ in CURRENT]) #game.split('-')])
 .pipe(lambda df_: df_.loc[df_['pos'] == 'DST'])
 # .sort_values('opp')
 .groupby('opp')
 ['fpts']
 .agg([percentile(25), 'median', percentile(75), 'max', 'sum'])
 .sort_values('median', ascending=False)
)

,floor,median,ceiling,max,sum
opp,,,,,
DAL,0.0,3.0,5.00,16.0,23.0
LAC,2.0,2.5,3.25,4.0,11.0


In [22]:
(pd
 .concat([analyze_kickers(against_) for against_ in game.split('-')])
 .groupby('opp')
 ['fpts']
 .agg(['median', 'mean'])
)

,median,mean
opp,,
DAL,5.0,5.75
LAC,7.5,8.00


In [ ]:
def player_profile(name: str):
    df_ = dataprep.fpts_df

    pos = df_.loc[df_['name'] == name, 'pos'].drop_duplicates().item()

    pos_stats = {
        'QB': ['pass_cmp', 'pass_att', 'pass_yds', 'pass_td', 'pass_rating', 'rush_att', 'rush_yds', 'rush_td'],
        'RB': ['rush_att', 'rush_yds', 'rush_td', 'targets', 'rec', 'rec_yds', 'rec_td'],
        'WR': ['targets', 'rec', 'rec_yds', 'rec_td'],
        'TE': ['targets', 'rec', 'rec_yds', 'rec_td'],
    }

    stats = ['week', 'name', 'pos', 'team', 'opp', 'fpts'] + pos_stats.get(pos, []) + ['home', 'score', 'winner', 'spread', 'total']
    
    return (df_
            .loc[df_['name'] == name]
            .sort_values('week')
            [stats]
            .assign(**{c: lambda df_, c=c:df_[c].astype('int') for c in pos_stats.get(pos, [])})
           )

def compare_teammates(*names):
    return (pd
            .concat([player_profile(name) for name in names])
            .sort_values('week', ascending=False)
            .fillna(0.0)
           )

In [ ]:
compare_teammates('Treylon Burks', 'Chris Moore', 'DeAndre Hopkins')

In [53]:
game = 'DAL-LAC'

In [54]:
awayteam, hometeam = game.split('-')

In [55]:
away = df.loc[df['team'] == awayteam]
home = df.loc[df['team'] == hometeam]

vs_away = df.loc[df['opp'] == awayteam]
vs_home = df.loc[df['opp'] == hometeam]

In [ ]:
perspective_filters = [['fpts', '>', 0.0]]

In [82]:
(home
 .pipe(lambda df_: df_.loc[df_['pos'].isin(['WR', 'TE'])])
 .sort_values(['pos', 'week', 'snap_total'], ascending=False)
 [['name', 'team', 'opp', 'pos', 'week', 'targets', 'rec', 'rec_yds', 'rec_td', 'snap_total', 'snap_percent', 'fpts']]
)

,name,team,opp,pos,week,targets,rec,rec_yds,rec_td,snap_total,snap_percent,fpts
965,Joshua Palmer,LAC,LV,WR,4,8.0,3.0,77.0,0.0,56,0.86,10.70
1042,Keenan Allen,LAC,LV,WR,4,5.0,3.0,32.0,1.0,49,0.75,12.20
1377,Quentin Johnston,LAC,LV,WR,4,3.0,1.0,18.0,0.0,33,0.51,2.80
516,Derius Davis,LAC,LV,WR,4,2.0,2.0,16.0,0.0,12,0.18,8.70
1041,Keenan Allen,LAC,MIN,WR,3,20.0,18.0,215.0,0.0,59,0.89,48.46
1274,Mike Williams,LAC,MIN,WR,3,8.0,7.0,121.0,1.0,44,0.67,28.10
964,Joshua Palmer,LAC,MIN,WR,3,7.0,4.0,66.0,1.0,40,0.61,15.60
1376,Quentin Johnston,LAC,MIN,WR,3,3.0,2.0,10.0,0.0,16,0.24,3.00
515,Derius Davis,LAC,MIN,WR,3,1.0,1.0,3.0,0.0,6,0.09,1.30
1040,Keenan Allen,LAC,TEN,WR,2,10.0,8.0,111.0,2.0,64,0.94,34.10


In [ ]:
# perspective.PerspectiveWidget(away, filter=perspective_filters)

In [ ]:
# perspective.PerspectiveWidget(vs_away, filter=perspective_filters)

In [ ]:
# perspective.PerspectiveWidget(home, filter=perspective_filters)

In [ ]:
# perspective.PerspectiveWidget(vs_home, filter=perspective_filters)

In [ ]:
pd.options.display.max_columns = 50

In [ ]:
vs_away.sort_values(['pos', 'fpts', 'targets', 'snap_total'], ascending=[True] + [False]*3 )

In [ ]:
vs_home.sort_values(['pos', 'opp', 'fpts', 'targets', 'snap_total'], ascending=[True] + [False]*4 )

In [56]:
def positions_against(opp_: str):
    target_depths = ['QB1', 'WR1', 'WR2', 'WR3', 'TE1', 'TE2', 'RB1', 'RB2']
    vs = (dataprep.load_analysis_data()
          .pipe(lambda df_: df_.loc[(df_['opp'] == opp_) & (df_['depth'].isin(target_depths))])
         )

    stats = ['targets', 'rec', 'rec_td', 'rush_yds', 'rush_td', 'fpts']
    print(f'Positions against {opp_}\n\n')
    return (vs
            .groupby('depth')
            [stats]
            .agg(['sum'])
            .round(2)
           )

In [57]:
positions_against(awayteam)

Positions against DAL




,targets,rec,rec_td,rush_yds,rush_td,fpts
,sum,sum,sum,sum,sum,sum
depth,,,,,,
QB1,0.0,0.0,0.0,141.0,0.0,64.7
RB1,11.0,7.0,0.0,197.0,2.0,42.9
RB2,8.0,6.0,0.0,76.0,0.0,14.9
TE1,22.0,17.0,3.0,0.0,0.0,56.0
TE2,5.0,3.0,0.0,0.0,0.0,4.5
WR1,30.0,16.0,2.0,0.0,0.0,53.3
WR2,18.0,10.0,0.0,84.0,1.0,34.4
WR3,12.0,6.0,0.0,0.0,0.0,20.0


In [58]:
positions_against(hometeam)

Positions against LAC




,targets,rec,rec_td,rush_yds,rush_td,fpts
,sum,sum,sum,sum,sum,sum
depth,,,,,,
QB1,0.0,0.0,0.0,27.0,1.0,82.86
RB1,24.0,18.0,0.0,268.0,3.0,76.90
RB2,4.0,4.0,0.0,49.0,0.0,10.90
TE1,22.0,15.0,0.0,0.0,0.0,29.70
TE2,4.0,3.0,1.0,0.0,0.0,10.50
WR1,46.0,30.0,3.0,0.0,0.0,101.90
WR2,20.0,15.0,1.0,0.0,0.0,39.80
WR3,16.0,9.0,1.0,0.0,0.0,31.60


In [59]:
def positions_against_fpts_breakdown(opp_: str):
    target_depths = ['WR1', 'WR2', 'WR3', 'TE1', 'TE2', 'TE3', 'RB1', 'RB2']
    vs = df.loc[(df['opp'] == opp_) & (df['depth'].isin(target_depths))]

    stats = ['fpts']

    print(f'FPTS breakdown against {opp_}\n\n')
    
    return (vs
            .groupby('depth')
            [stats]
            .agg(['min', percentile(25), 'median', percentile(75), 'max', 'count'])
            .round(2)
           )

In [60]:
positions_against_fpts_breakdown(awayteam)

FPTS breakdown against DAL




fpts                                 
       min floor median ceiling   max count
depth                                      
RB1    0.9  1.60    6.0   14.80  19.6     5
RB2    0.2  0.95    2.7    5.48   9.3     4
TE1    2.6  6.60    9.1   10.00  27.7     5
TE2    1.1  1.15    1.2    1.70   2.2     3
TE3    3.7  3.70    3.7    3.70   3.7     1
WR1    4.5  5.60    9.8   16.30  17.1     5
WR2    2.4  3.83    7.9   12.68  16.2     4
WR3    0.0  1.20    2.9    5.30  10.6     5

In [61]:
positions_against_fpts_breakdown(hometeam)

FPTS breakdown against LAC




fpts                                  
        min  floor median ceiling   max count
depth                                        
RB1    13.0  16.38  18.00   20.85  27.9     4
RB2     3.4   4.43   5.45    6.48   7.5     2
TE1     7.4   7.45   7.50   11.15  14.8     3
TE2     2.1   3.68   5.25    6.82   8.4     2
TE3     0.0   0.00   0.00    0.00   0.0     1
WR1     8.0  13.62  23.20   35.05  47.5     4
WR2     5.3   9.72  11.35   11.58  11.8     4
WR3     3.2   6.20   8.90   10.60  10.6     4

In [ ]:
def target_share(team: str):

    tdf = df.loc[df['team'] == team]

    return (tdf
            .groupby('name')
            ['targets']
            .agg(['sum', 'mean'])
            .set_axis(['total', 'avg'], axis=1)
            .pipe(lambda df_: df_.loc[df_['total'] > 0])
            .sort_values('total')
            .plot
            .barh(
                figsize=(9,6),
                title=f'{team} Targets'
            )
           )

In [ ]:
def snapcounts_share(team: str):

    tdf = df.loc[df['team'] == team]

    return (tdf
            .groupby('name')
            ['snap_total']
            .agg(['sum', 'mean'])
            .set_axis(['total', 'avg'], axis=1)
            .pipe(lambda df_: df_.loc[df_['total'] > 0])
            .sort_values('total')
            .plot
            .barh(
                figsize=(9,6),
                title=f'{team} Snapcounts'
            )
           )

In [ ]:
target_share(awayteam);

In [ ]:
snapcounts_share(awayteam);

In [ ]:
target_share(hometeam);

In [ ]:
snapcounts_share(hometeam);

In [ ]:
# Perspective automatically makes defense as a date

# df['opp-defense'] = df['opp-defense'].map(lambda d: f'vs{d}')

# perspective.PerspectiveWidget(df)

In [ ]:
defenses = pd.read_csv('../data/2023-2024/defenses.csv')
breakdown = {def_: len(defenses.loc[defenses['defense'] == def_]) for def_ in defenses['defense'].drop_duplicates()}
breakdown

In [ ]:
df_34 = df.loc[df['opp-defense'] == 'vs3-4']
df_43 = df.loc[df['opp-defense'] == 'vs4-3']

In [ ]:
def pos_vs34(pos: str, **kwargs):
    print(f'{pos} vs. 3-4 Defense\n\n')
    
    pos_stats = {
        'QB': ['pass_yds', 'pass_td', 'rush_yds', 'rush_td'],
        'RB': ['targets', 'rec', 'rush_yds', 'rush_td'],
        'WR': ['targets', 'rec', 'rec_yds', 'rec_td'],
        'TE': ['targets', 'rec', 'rec_yds', 'rec_td']
    }
    stats = ['fpts'] + pos_stats[pos] + kwargs.get('stats', [])
    agg_by = ['mean', 'sum']
    return (df_34
            .loc[(df['pos'] == pos) & (df['fpts'] > 0.0)]
            .groupby('depth')
            [stats]
            .agg(['mean', 'sum'])
            .set_axis([f'{agg_}-{stat_}' for stat_ in stats for agg_ in agg_by], axis=1)
            .round(2)
           )

def pos_vs43(pos: str, **kwargs):
    print(f'{pos} vs. 4-3 Defense\n\n')
    pos_stats = {
        'QB': ['pass_yds', 'pass_td', 'rush_yds', 'rush_td'],
        'RB': ['targets', 'rec', 'rush_yds', 'rush_td'],
        'WR': ['targets', 'rec', 'rec_yds', 'rec_td'],
        'TE': ['targets', 'rec', 'rec_yds', 'rec_td']
    }
    stats = ['fpts'] + pos_stats[pos] + kwargs.get('stats', [])
    agg_by = ['mean', 'sum']
    return (df_43
            .loc[(df['pos'] == pos) & (df['fpts'] > 0.0)]
            .groupby('depth')
            [stats]
            .agg(['mean', 'sum'])
            .set_axis([f'{agg_}-{stat_}' for stat_ in stats for agg_ in agg_by], axis=1)
            .round(2)
           )

In [ ]:
# pos_vs34('QB')

In [ ]:
pos_vs43('QB')

In [ ]:
pos_vs34('RB')

In [ ]:
pos_vs43('RB')

In [ ]:
pos_vs34('WR')

In [ ]:
pos_vs43('WR')

In [ ]:
pos_vs34('TE')

In [ ]:
pos_vs43('TE')